In [ ]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
def add_date_time(df):
    dts = []
    for idx in df.index:
        dts.append( datetime(
            df.loc[idx, 'year'],
            df.loc[idx, 'month'],
            df.loc[idx, 'day'],
            df.loc[idx, 'hour']-1, 
            0 # minutes
        ))
    df['date_time'] = dts
    return df

In [ ]:
# Shows simplified, clean results     
def demand_plot3(df, dt_shifted, wind, solar, wind_cap, solar_cap, title, steps=1):
    plt.close()
    fig, ax = plt.subplots(figsize=(7,6))
    matplotlib.rcParams.update({'font.size': 16})
    #ax.set_xlabel('Hour')
    ax.set_ylabel('Power (MW)')
    ax.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))

    
    #mean_vre = np.mean(vre[vre_name.lower()])
    #mean_vre = 1.0
    #mean_dem = np.mean(df['Cleaned Demand (MW)'])
    #factor = factor*(mean_dem/mean_vre)
    
    print(len(df.index))
    print(len(wind.index))
    print(len(solar.index))
    
    mean = np.mean(df['demand (MW)'])
    # Find % wind and solar
    pct_wind = np.sum(wind['wind capacity']*wind_cap)/len(df.index)
    print(pct_wind)
    pct_solar = np.sum(solar['solar capacity']*solar_cap)/len(df.index)
    print(pct_solar)
    
    ax.plot(dt_shifted, df['demand (MW)'], 'k-', label='demand', linewidth=4.0)
    if steps > 1:
        ax.fill_between(dt_shifted,
                    df['demand (MW)']-wind['wind capacity']*wind_cap*mean, df['demand (MW)'],
                    color='blue', label='wind', alpha=0.5)
    if steps > 2:
        ax.fill_between(dt_shifted,
                    df['demand (MW)']-wind['wind capacity']*wind_cap*mean-solar['solar capacity']*solar_cap*mean,
                    df['demand (MW)']-wind['wind capacity']*wind_cap*mean,
                    color='orange', label='solar', alpha=0.5)
    if steps > 3:
        ax.plot(dt_shifted, df['demand (MW)']-wind['wind capacity']*wind_cap*mean-solar['solar capacity']*solar_cap*mean,
                    'k--', label='demand - VRE', linewidth=4.0)
    
    # Reorder legend labels as I want
    handles, labels = ax.get_legend_handles_labels()
    h_leg = [None for _ in range(4)]
    order = ['demand', 'wind', 'solar', 'demand - VRE']
    for h, l in zip(handles, labels):
        idx = order.index(l)
        h_leg[idx] = h
        
    plt.legend(h_leg, order, prop={'size': 17}, loc='upper center')
    #ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=mdates.MO))
    ax.xaxis.set_major_locator(mdates.DayLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    ax.xaxis.set_minor_locator(mdates.HourLocator())
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=45 )
    #ax.xaxis.set_minor_locator(mdates.DayLocator())
    ax.set_ylim(0, 100000)
    plt.tight_layout()
    #plt.grid()
    plt.savefig(f"windGen{pct_wind}_solarGen{pct_solar}_step{steps}.png")
    return


reg = 'ERCOT'
fPath = 'ERCOT_demand_unnormalized_short.csv'

df = pd.read_csv(fPath, header=6)
df = add_date_time(df)
solar_df = pd.read_csv('ERCOT_solar_top25pct_unnormalized_short.csv', header=6)
solar_df = add_date_time(solar_df)
wind_df = pd.read_csv('ERCOT_wind_top25pct_unnormalized_short.csv', header=6)
wind_df = add_date_time(wind_df)

dt_shifted = df['date_time'] - timedelta(hours=6)
dem = df.loc[4005:4125]
dt = dt_shifted.loc[4005:4125]
wind = wind_df.loc[4005:4125]
solar = solar_df.loc[4005:4125]
wind_CF = np.mean(wind_df['wind capacity'])
solar_CF = np.mean(solar_df['solar capacity'])
#print(wind_CF, solar_CF)

wind_cap = 0.5*1.9/1.09
solar_cap = 0.5*1.431
for step in range(1, 5):
    demand_plot3(dem, dt, wind, solar, wind_cap, solar_cap, reg, step)